In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [15]:
meta = pd.read_csv( '/kaggle/input/the-movies-dataset/movies_metadata.csv' )


meta = meta[ ['id', 'original_title', 'original_language', 'genres'] ]

meta = meta.rename(columns={'id':'movieId','original_title': 'title','original_language': 'language'})

meta = meta.loc[meta['language'] == 'en',:] # 데이터 조회, 필터링

# meta.loc[3,'title']
# meta.loc[3,:]
# meta['title']
meta['movieId'] = pd.to_numeric(meta['movieId'])
meta['genres'] = meta['genres'].apply(str_to_set)
meta

In [12]:
keywords = pd.read_csv('/kaggle/input/the-movies-dataset/keywords.csv')
keywords = keywords.rename(columns={'id':'movieId'})

keywords.loc[0,'keywords']

def str_to_set(s):
    return set(item['name'] for item in eval(s))
#     genre_set = set()
#     for item in eval(s):
#         genre_set.add(item['name'])
#     return genre_set
    
keywords['keywords'] = keywords['keywords'].apply(str_to_set)

keywords

In [18]:
merged = pd.merge(meta,keywords, on = 'movieId', how = 'inner') # 둘 다 있는 id만 남겨둔다.교집합
# inner join 교집합
# outer join 합집합
# left join 왼쪽 집합
# right join 오른쪽 집합

In [33]:
dk = merged.loc[merged['title']=='The Dark Knight',:].iloc[0,:]
dkr = merged.loc[merged['title']=='The Dark Knight Rises',:].iloc[0,:]
toy = merged.loc[merged['title']=='Toy Story',:].iloc[0,:]

dk_set = dk.keywords | dk.genres
dkr_set = dkr.keywords | dkr.genres
toy_set = toy.keywords | toy.genres

len(dk_set & dkr_set)/len(dk_set | dkr_set)

def jaccard_similarity(set1, set2) :
    if len(set1 | set2) == 0 : return 0
    return len(set1 & set2)/len(set1 | set2)

a = jaccard_similarity(dk_set, dkr_set)
b = jaccard_similarity(dk_set, toy_set)
print(a,b)

In [47]:
ratings = pd.read_csv('/kaggle/input/the-movies-dataset/ratings_small.csv')
ratings = pd.merge(ratings[['userId','movieId','rating']], meta[['movieId', 'title']], on='movieId', how='inner')
ratings

# pearson correlation cofficient
# 두 영화의 평점 vector가 필요하다!




# 1970.01.01 부터 얼마나 시간이 흘렀는지 단위는 초
# from datetime import datetime
# datetime.fromtimestamp(0)

In [53]:
matrix = ratings.pivot_table(index= 'userId', columns='title',values='rating')

# 값들이 비어있을 떄 유리하다.
def pearson_similarity(a, b):
    a = a - a.mean() # 평균이 0이 되도록 하는 작업
    b = b - b.mean()
    denom = np.sqrt(np.sum(a ** 2) * np.sum(b ** 2))
    if denom != 0:
        return np.sum(a * b)/denom
    else:
        return 0

pn = matrix['Prom Night']
dk = matrix['The Dark Knight']

pearson_similarity(pn,dk)

In [61]:
alpha = 0.5
k = 10

# The Dark Knight와 모든 영화 사이의 Pearson, Jaccard 유사도 구하기
input_title = 'The Dark Knight'
input_vector = matrix[input_title]

input_meta = merged.loc[merged['title']=='The Dark Knight',:].iloc[0,:]
input_set = input_meta['genres'] | input_meta['keywords']

results = []


for this_title in matrix.columns : 
    this_vector = matrix[this_title]
    
    this_meta = merged.loc[merged['title'] == this_title, :].iloc[0,:]
    this_set = this_meta['genres'] | this_meta['keywords']
    
    jaccard = jaccard_similarity(input_set, this_set)
    pearson = pearson_similarity(input_vector, this_vector)
    
    score = alpha * jaccard + (1-alpha) * pearson
    
    results.append((this_title, jaccard, pearson, score))

results.sort(key = lambda item:item[3], reverse = True)
results[:10]


In [62]:
pd.DataFrame(results[:10], columns=['title','jaccard','pearson','score'])

In [77]:
def find_similar_movies(input_title, matrix, merged, alpha=0.5, k=10):
    input_vector = matrix[input_title]

    input_meta = merged.loc[merged['title']==input_title,:].iloc[0,:]
    input_set = input_meta['genres'] | input_meta['keywords']

    results = []

    for this_title in matrix.columns : 
        this_vector = matrix[this_title]

        this_meta = merged.loc[merged['title'] == this_title, :].iloc[0,:]
        this_set = this_meta['genres'] | this_meta['keywords']

        jaccard = jaccard_similarity(input_set, this_set)
        pearson = pearson_similarity(input_vector, this_vector)

        score = alpha * jaccard + (1-alpha) * pearson

        results.append((this_title, jaccard, pearson, score))
        
    results.sort(key = lambda item:item[3], reverse = True)
        
    return pd.DataFrame(results[:k], columns=['title','jaccard','pearson','score'])


In [78]:
find_similar_movies('Rocky III', matrix, merged, 0.3, 20)

In [79]:
# 평가데이터 부족할 경우
find_similar_movies("Love Is a Ball", matrix, merged, 0.3, 20)